In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD module
from Mongopython import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Adonis"

# Connect to the MongoDB database using the AnimalShelter class
db = AnimalShelter(username, password)

# Fetch initial data from the database
df = pd.DataFrame.from_records(db.read({}))

# Remove MongoDB ObjectID column for compatibility with the data table
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png'  # replace with your image file path
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout with logo and user identifier
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100px'})),
    html.Center(html.B(html.H1('Grazioso Salvare - Animal Rescue Dashboard'))),
    html.Hr(),
    html.Div("Created by: Caleb Leavell"),  # Add your unique identifier here
    
    # Add filtering options for rescue types
    html.Div([
        html.Label('Select Rescue Type:'),
        dcc.Dropdown(
            id='filter-type',
            options=[ 
                {'label': 'All Types', 'value': 'All Types'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
            ],
            value='Water Rescue',  # Default filter option
            clearable=False,
            style={'width': '50%'}
        )
    ]),

    html.Hr(),
    
    # Interactive DataTable for filtered results
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'height': '300px', 'overflowY': 'auto'},
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
    ),
    
    html.Br(),
    html.Hr(),

    # Layout for charts and maps side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update DataTable based on the rescue type filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # MongoDB queries based on rescue type
    if filter_type == 'Water Rescue':
        filtered_data = db.read({"breed": {"$in": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]}})
    elif filter_type == 'Mountain or Wilderness Rescue':
        filtered_data = db.read({"breed": {"$in": ["German Shepherd", "Belgian Malinois"]}})
    elif filter_type == 'Disaster or Individual Tracking':
        filtered_data = db.read({"breed": {"$in": ["Bloodhound", "Beagle"]}})
    else:
        filtered_data = db.read({})  # Default query to return all data

    # Convert filtered data into DataFrame and dictionary format
    df_filtered = pd.DataFrame.from_records(filtered_data)
    df_filtered.drop(columns=['_id'], inplace=True)  # Remove the '_id' column for table display
    return df_filtered.to_dict('records')

# Callback to update breed distribution chart based on DataTable selection
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(view_data):
    if view_data is None:
        return "No data available"
    
    # Convert view_data to DataFrame
    dff = pd.DataFrame.from_dict(view_data)
    
    # Generate pie chart for breed distribution
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Callback to update the geo-location map based on selected DataTable row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(view_data, index):
    if view_data is None or index is None or len(index) == 0:
        return "No location available"

    # Convert view_data to DataFrame
    dff = pd.DataFrame.from_dict(view_data)
    
    # Get the selected row
    selected_row = index[0] if index else 0

    # Default to Austin, TX if location not found
    try:
        lat, lon = dff.iloc[selected_row]['location_lat'], dff.iloc[selected_row]['location_lon']
    except (IndexError, KeyError):
        lat, lon = 30.75, -97.48

    # Return map centered on the selected animal's location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[selected_row]['breed']),
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[selected_row]['name'])])
            ])
        ])
    ]

#############################################
# Run the app
#############################################
if __name__ == '__main__':
    app.run_server(debug=True)

Username: aacuser, Password: Adonis
Dash app running on http://127.0.0.1:8257/
